## 뱀

### Problem
- 뱀이 나와서 기어다니는데, 사과를 먹으면 뱀 길이가 늘어나고, 벽 또는 자신의 몸과 부딪히면 게임이 끝남
- N x N 보드에서 진행되며, 몇몇 칸에는 사과가 놓여져 있음
- 보드의 가장자리는 벽으로 되어있으며, 뱀은 맨 위 좌측에서 시작하고 길이는 1이며 오른쪽을 향함
- 뱀은 매 초마다 아래의 규칙을 따름
 - 먼저 뱀은 몸 길이를 늘려 머리를 다음 칸에 위치시킴
 - 만약 이동한 칸에 사과가 있다면, 그 칸에 있던 사과가 없어지고 꼬리는 움직이지 ㅇ낳음
 - 만약 이동한 칸에 사과가 없다면, 몸길이를 줄여서 꼬리가 위치한 칸을 비워줌
- **사과의 위치와 뱀의 이동경로가 주어질 때, 이 게임이 몇 초에 끝나는지 계산**

### Input
- 첫째 줄에 보드의 크기 N이 주어짐. (2 <= N <= 100)
- 다음 줄에 사과의 개수 K가 주어짐 (0 <= K <= 100)
- 다음 K개의 줄에는 사과의 위치가 주어짐, 첫 번재 정수는 행, 두 번째 정수는 열 위치를 의미
 - 1열 1행에는 사과가 없음
- 다음 줄에는 뱀의 방향 변환 횟수 L이 주어짐 (1 <= L <= 100)
- 다음 L개의 줄에는 뱀의 방향 변환 정보가 주어짐, 정수 X와 문자 C로 이루어짐
 - X: X초가 끝난 뒤에 (0 < X <= 10,000)
 - C: 'L' 왼쪽 'D' 오른쪽으로 90도 방향 회전

### Output
- 게임이 몇초에 끝나는지 출력

### Approch
- NxN 매트릭스를 생성한 후, 해당 위치에 맞게 사과를 마크함
- 뱀은 Array를 이용한 Circular Queue로 구성을 하며, 이동 시엔 Head로는 Enqueue / Tail로는 Dequeue
 - 매번 이동을 할 때마다 벽인지 자기 자신에 부딪혔는지 체크
 - 사과를 먹었을 경우에는 Head에 다음 위치만 추가
- 총 시간 복잡도는 O(XK)
 - 매 초마다 이동하며 O(X)
 - 이동할 때마다 벽에 부딪혔는지 체크 O(1) OR 자기 자신에 부딪혔는지 체크 O(K) **최대 뱀의 길이는 사과의 개수**

In [38]:
def get_int():
    num = input()
    return int(num)

In [39]:
def get_int_list():
    num_str = input()
    num_list = [int(num) for num in num_str.split()]
    return num_list

In [40]:
def get_int_ch_list():
    buf_str = input()
    buf = buf_str.split()
    list = [int(buf[0]), buf[1]]
    return list

In [41]:
class CircularQueue:
    def __init__(self, k: int):
        self.k = k
        self.q = [None] * k
        self.head = 0
        self.tail = 0

    def enqueue(self, value: list) -> bool:
        if self.q[self.tail] is None:
            self.q[self.tail] = value
            self.tail = (self.tail + 1) % self.k
            return True
        else:
            return False

    def dequeue(self) -> bool:
        if self.q[self.head] is not None:
            self.q[self.head] = None
            self.head = (self.head + 1) % self.k
            return True
        else:
            return False

    def front(self) -> list:
        return self.q[self.head] if self.q[self.head] is not None else None

    def rear(self) -> list:
        return self.q[self.tail - 1] if self.q[self.tail - 1] is not None else None

    def is_empty(self) -> bool:
        return self.head == self.tail and self.q[self.head] is None

    def is_full(self) -> bool:
        return self.head == self.tail and self.q[self.head] is not None
    
    def is_gameover(self, next_head: list) -> bool:
        if self.tail >= self.head:
            for i in range(self.head, self.tail):
                if self.q[i][0] == next_head[0] and self.q[i][1] == next_head[1]:
                    return True
        else:
            for i in range(self.tail, self.k):
                if self.q[i][0] == next_head[0] and self.q[i][1] == next_head[1]:
                    return True
            for i in range(0, self.head):
                if self.q[i][0] == next_head[0] and self.q[i][1] == next_head[1]:
                    return True
                
        return False    

In [54]:
def solution(N: int, K: int, apples: list, L: int, moves: list) -> int:
    
    # 1. Make game map as numpy array.
    game_map = [[0 for _ in range(N)] for _ in range(N)]
    
    # 2. Update apple.
    for index in apples:
        game_map[index[0] - 1][index[1] - 1] = 1
    
    # 3. Generate snake by queue.
    snake = CircularQueue(101)
    snake.enqueue([0, 0])

    # 4. Start game, increase 1 sec and move snake and check
    direction = 0
    moves_index = 0
    for X in range(1, 10001):
        next_head = [-1, -1]
        curr_head = snake.rear()

        # 4-1. Get position next snake head.
        if direction == 0: # Right
            next_head[0] = curr_head[0]
            next_head[1] = curr_head[1] + 1
        elif direction == 1: # Up
            next_head[0] = curr_head[0] - 1
            next_head[1] = curr_head[1]
        elif direction == 2: # Left
            next_head[0] = curr_head[0]
            next_head[1] = curr_head[1] - 1
        elif direction == 3: # Down
            next_head[0] = curr_head[0] + 1
            next_head[1] = curr_head[1]
        else:
            # MUST BE NOT REACHABLE
            return -1
        
        # 4-2. Check approching barrier
        if next_head[0] == -1 or next_head[0] == N:
            return X
        if next_head[1] == -1 or next_head[1] == N:
            return X
            
        # 4-3. Check approching snake itself.
        if snake.is_gameover(next_head):
            return X
        
        # 4-4. Move snake
        snake.enqueue(next_head)
        if game_map[next_head[0]][next_head[1]] == 0:
            snake.dequeue()
        else:
            game_map[next_head[0]][next_head[1]] = 0
        
        #  4-5. Update direction
        if moves_index < L and moves[moves_index][0] == X:
            if moves[moves_index][1] == 'L':
                direction = (direction + 1) % 4
            elif moves[moves_index][1] == 'D':
                direction = (direction - 1) % 4
            moves_index += 1

    return -1

In [47]:
def main():
    N = get_int()
    K = get_int()
    apples = []
    for _ in range(K):
        apples.append(get_int_list())
    L = get_int()
    moves = []
    for _ in range(L):
        moves.append(get_int_ch_list())
    print(solution(N, K, apples, L, moves))

In [55]:
N = 6
K = 3
apples = [[3, 4], [2, 5], [5, 3]]
L = 3
moves = [[3, 'D'], [15, 'L'], [17, 'D']]

In [50]:
N = 10
K = 4
apples = [[1, 2], [1, 3], [1, 4], [1, 5]]
L = 4
moves = [[8, 'D'], [10, 'D'], [11, 'D'], [13, 'L']]

In [57]:
N = 10
K = 5
apples = [[1, 5], [1, 3], [1, 2], [1, 6], [1, 7]]
L = 4
moves = [[8, 'D'], [10, 'D'], [11, 'D'], [13, 'L']]

In [58]:
solution(N, K, apples, L, moves)

13